In [ ]:
# agent_feedback_response.py

! pip install llama-index llama-index-core llama-index-embeddings-huggingface llama-index-llms-groq transformers

from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core.chat_engine import SimpleChatEngine

from transformers import pipeline

MODEL_NAME = "llama-3.3-70b-versatile"
API_KEY = "" # Replace with your actual API key
EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
KNOWLEDGE_BASE = "./budget_data/"
CHUNK_SIZE = 512
CHUNK_OVERLAP = 20
OUTPUT_TOKENS = 512


# Load sentiment analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english"
)


def get_llm(model_name, api_key):
    return Groq(model=model_name, api_key=api_key, temperature=0.5)

def initialize_settings():
    Settings.llm = get_llm(MODEL_NAME, API_KEY)
    Settings.embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
    Settings.num_output = OUTPUT_TOKENS
    Settings.node_parser = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)


# define tools
def multiply(a, b):
    return a * b

def add(a, b):
    return a + b

multiplication_tool = FunctionTool.from_defaults(fn=multiply)
addition_tool = FunctionTool.from_defaults(fn=add)

initialize_settings()


agent = SimpleChatEngine.from_defaults(
    tools=[multiplication_tool, addition_tool],
    llm=Settings.llm,
    verbose=True,
)


def generate_response(review_text, sentiment_label):
    prompt = f"""
    A customer left the following {sentiment_label} review:

    "{review_text}"

    Write a short, polite, and personalized response from the restaurant.
    """

    res = agent.chat(prompt)
    return res




def get_sentiment(review_text):
    sentiment_result = sentiment_model(review_text)[0]
    sentiment = sentiment_result['label'].lower()
    return sentiment

# Example usage
if __name__ == "__main__":
    review = "The food was great."
    sentiment= get_sentiment(review)

    print(f"Detected Sentiment: {sentiment}")
    response = generate_response(review, sentiment)

    print(f"Response: {response} ")






Device set to use cpu


Detected Sentiment: negative
Response: Here's a potential response:

"Dear [Customer's Name], 

Thank you for taking the time to share your feedback about your recent visit to our restaurant. We're thrilled to hear that you enjoyed your meal, but sorry to hear that our service didn't quite meet your expectations. We apologize for the delay and appreciate your patience. We'll do our best to improve our service speed while maintaining the high quality of our food. We hope to have the opportunity to serve you again soon and provide a more well-rounded experience.

Best regards, [Restaurant Name]" 
